In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import datetime
from dotenv import load_dotenv
import httpx
from itertools import cycle
import json
from lib.chat_completions import get_chat_completion
from lib.tokenizer import Tokenizer
from lib.utils import black_print
import openai
import os
import polars as pl
from pydantic import BaseModel
import torch
import tqdm


load_dotenv()

df = (
    pl.read_parquet("./data/corbt/connections-games/*.parquet")
    .sort("createdAt")
    .cast({"id": pl.Int64})
    .rename({"startingBoard": "starting_board"})
)
df

INFO 02-11 19:31:42 __init__.py:190] Automatically detected platform cpu.


id,board,createdAt,name,starting_board
i64,str,"datetime[μs, UTC]",str,str
1,"""{""WET WEATHER"": {""level"": 0, ""…",2023-06-12 00:00:00 UTC,"""Connections #1""","""[[""SNOW"", ""LEVEL"", ""SHIFT"", ""K…"
2,"""{""FOOTWEAR"": {""level"": 0, ""mem…",2023-06-13 00:00:00 UTC,"""Connections #2""","""[[""PUMP"", ""FOOT"", ""TIME"", ""SEA…"
3,"""{""FACIAL FEATURES"": {""level"": …",2023-06-14 00:00:00 UTC,"""Connections #3""","""[[""AMIGO"", ""MOUTH"", ""LAB"", ""ST…"
4,"""{""SNEAKER BRANDS"": {""level"": 0…",2023-06-15 00:00:00 UTC,"""Connections #4""","""[[""DUST"", ""CATS"", ""SPIDER"", ""C…"
5,"""{""STREAMING SERVICES"": {""level…",2023-06-16 00:00:00 UTC,"""Connections #5""","""[[""MUSTARD"", ""TARTAR"", ""PLUM"",…"
…,…,…,…,…
596,"""{""SMALL AMOUNT OF FOOD TO TRY""…",2025-01-27 00:00:00 UTC,"""Connections #596""","""[[""OLIVE"", ""TYPEWRITER"", ""EXTR…"
597,"""{""FUNNY PERSON"": {""level"": 0, …",2025-01-28 00:00:00 UTC,"""Connections #597""","""[[""DEMON"", ""LAUGH"", ""JOKER"", ""…"
598,"""{""OWNED"": {""level"": 0, ""member…",2025-01-29 00:00:00 UTC,"""Connections #598""","""[[""WICKED"", ""GINGERBREAD"", ""FU…"


In [3]:
class ConnectionGroup(BaseModel):
    level: int
    members: list[str]


class ConnectionGame(BaseModel):
    board: dict[str, ConnectionGroup]
    starting_board: list[list[str]]
    createdAt: datetime.datetime
    id: int
    name: str


games: list[ConnectionGame] = [
    ConnectionGame(
        board={
            group_name: ConnectionGroup(
                level=group_contents["level"],
                members=group_contents["members"],
            )
            for group_name, group_contents in json.loads(d["board"]).items()
        },
        starting_board=json.loads(d["starting_board"]),
        createdAt=d["createdAt"],
        id=d["id"],
        name=d["name"],
    )
    for d in df.to_dicts()
]
games

[ConnectionGame(board={'WET WEATHER': ConnectionGroup(level=0, members=['HAIL', 'RAIN', 'SLEET', 'SNOW']), 'NBA TEAMS': ConnectionGroup(level=1, members=['BUCKS', 'HEAT', 'JAZZ', 'NETS']), 'KEYBOARD KEYS': ConnectionGroup(level=2, members=['OPTION', 'RETURN', 'SHIFT', 'TAB']), 'PALINDROMES': ConnectionGroup(level=3, members=['KAYAK', 'LEVEL', 'MOM', 'RACECAR'])}, starting_board=[['SNOW', 'LEVEL', 'SHIFT', 'KAYAK'], ['HEAT', 'TAB', 'BUCKS', 'RETURN'], ['JAZZ', 'HAIL', 'OPTION', 'RAIN'], ['SLEET', 'RACECAR', 'MOM', 'NETS']], createdAt=datetime.datetime(2023, 6, 12, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), id=1, name='Connections #1'),
 ConnectionGame(board={'FOOTWEAR': ConnectionGroup(level=0, members=['BOOT', 'LOAFER', 'PUMP', 'SNEAKER']), 'UNITS OF LENGTH': ConnectionGroup(level=1, members=['FOOT', 'LEAGUE', 'MILE', 'YARD']), 'MAGAZINES': ConnectionGroup(level=2, members=['ESSENCE', 'PEOPLE', 'TIME', 'US']), 'LETTER HOMOPHONES': ConnectionGroup(level=3, members=['ARE', 'QUEUE', 'SEA

In [4]:
openai_client = openai.AsyncOpenAI()
openrouter_client = openai.AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY")
)
fireworks_client = openai.AsyncOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
    http_client=openai.DefaultAsyncHttpxClient(
        limits=httpx.Limits(
            max_connections=1_000,
            max_keepalive_connections=400,
        )
    )
)
chat_completion = await get_chat_completion(
    fireworks_client,
    messages=[
        {
            "role": "user",
            "content": "Hello, world!",
        }
    ],
    model="accounts/fireworks/models/deepseek-r1",
    logprobs=True,
    top_logprobs=5,
)
# # on_chunk=lambda chunk, _: print(getattr(chunk.choices[0].delta, "reasoning", "") or "", end="", flush=True),
# on_chunk=lambda chunk, _: print(chunk.choices[0].delta.content or "", end="", flush=True),
# )
black_print(chat_completion)

ChatCompletion(
    id="2775ed92-dc86-4a04-9d0f-35860a569c5b",
    choices=[
        Choice(
            finish_reason="stop",
            index=0,
            logprobs=ChoiceLogprobs(
                content=[
                    ChatCompletionTokenLogprob(
                        token="<think>",
                        bytes=[60, 116, 104, 105, 110, 107, 62],
                        logprob=-1.156e-05,
                        top_logprobs=[
                            TopLogprob(
                                token="<think>",
                                bytes=[60, 116, 104, 105, 110, 107, 62],
                                logprob=-1.156e-05,
                                token_id=128798,
                            ),
                            TopLogprob(
                                token="</think>",
                                bytes=[60, 47, 116, 104, 105, 110, 107, 62],
                                logprob=-11.61388779,
                                token

In [5]:
tokenizer = Tokenizer("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
tokenizer

INFO 02-11 19:31:45 config.py:2355] For macOS with Apple Silicon, currently bfloat16 is not supported. Setting dtype to float16.
WARNING 02-11 19:31:45 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-11 19:31:49 config.py:542] This model supports multiple tasks: {'classify', 'reward', 'score', 'embed', 'generate'}. Defaulting to 'generate'.
WARNING 02-11 19:31:49 arg_utils.py:1145] The model has a long context length (131072). This may cause OOM errors during the initial memory profiling phase, or result in low performance due to small KV cache space. Consider setting --max-model-len to a smaller value.
WARNING 02-11 19:31:49 config.py:678] Async output processing is not supported on the current platform type cpu.
WARNING 02-11 19:31:49 cpu.py:63] CUDA graph is not supported on CPU, fallback to the eager mode.
WARNING 02-11 19:31:49 cpu.py:78] Environment variable VLLM_CPU_KVCACHE_SPACE (GB) for CPU backend is not set, using 4 by default.
INFO 02-11 19:31:49 llm_engine

In [46]:
deepseek_tokenizer = Tokenizer("deepseek-ai/DeepSeek-R1")
deepseek_tokenizer

INFO 02-11 20:10:43 config.py:137] Replacing legacy 'type' key with 'rope_type'
INFO 02-11 20:10:47 config.py:2355] For macOS with Apple Silicon, currently bfloat16 is not supported. Setting dtype to float16.
WARNING 02-11 20:10:47 config.py:2386] Casting torch.bfloat16 to torch.float16.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 02-11 20:10:51 config.py:542] This model supports multiple tasks: {'classify', 'reward', 'score', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 02-11 20:10:51 importing.py:16] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 02-11 20:10:51 arg_utils.py:1145] The model has a long context length (163840). This may cause OOM errors during the initial memory profiling phase, or result in low performance due to small KV cache space. Consider setting --max-model-len to a smaller value.
WARNING 02-11 20:10:51 config.py:678] Async output processing is not supported on the current platform type cpu.
WARNING 02-11 20:10:51 fp8.py:52] Detected fp8 checkpoint. Please note that the format is experimental and subject to change.
WARNING 02-11 20:10:51 cpu.py:63] CUDA graph is not supported on CPU, fallback to the eager mode.
WARNING 02-11 20:10:51 cpu.py:78] Environment variable VLLM_CPU_KVCACHE_SPACE (GB) for CPU backend is not set, usi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
from openai.types.chat.chat_completion import ChatCompletion

# Prompts from https://github.com/lechmazur/nyt-connections
prompts = [
    "Find groups of four items that share something in common. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
    "Group words that share a common thread. There are four words for each common thread. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
    "This is a puzzle. Create four groups of four. Words in each group fit under a specific category. Some categories might be defined by their use of wordplay (palindromes, homophones, adding or dropping letters and words) rather than the literal meanings of the words on the cards. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
]
# eval_games = games[:436]
eval_games = games[:20]
client = fireworks_client
model = "accounts/fireworks/models/deepseek-r1"

pbar = tqdm.tqdm(total=len(eval_games))  # * len(prompts) * 2)
total_reward = 0
total_tokens = 0
generated_tokens = 0
exceptions: list[BaseException] = []


def update_postfix(additional_tokens: int = 0) -> None:
    global generated_tokens
    generated_tokens += additional_tokens
    pbar.set_postfix(
        {
            "reward": total_reward / max(1, pbar.n),
            "average tokens": total_tokens / max(1, pbar.n),
            "generated tokens": generated_tokens,
            "spend": f"${(generated_tokens/1_000_000) * 8:0.2f}",
            "exceptions": len(exceptions),
        }
    )


async def eval(
    game: ConnectionGame,
    prompt: str,
    lowercase: bool,
) -> tuple[float, ChatCompletion]:
    content = f"{prompt}\nWords:\n\n{"\n".join(word.lower() if lowercase else word for row in game.starting_board for word in row)}"
    chat_completion = await get_chat_completion(
        client,
        on_chunk=lambda chunk, _: update_postfix(
            additional_tokens=(
                len(chunk.choices[0].logprobs.content or [])
                if chunk.choices[0].logprobs
                else 0
            )
        ),
        log_results=False,
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model=model,
        max_tokens=2**17,
        logprobs=True,
        top_logprobs=5,
    )
    assistant_content = chat_completion.choices[0].message.content
    assert assistant_content is not None
    assistant_content = assistant_content.strip()
    groups = [
        frozenset(word.strip().upper() for word in line.split(","))
        for line in assistant_content.split("\n")[-len(game.board) :]
    ]
    reward = 0
    for _, group_contents in game.board.items():
        if set(group_contents.members) in groups:
            reward += 1 / len(game.board)
    # tokens = tokenizer.encode(
    #     [
    #         {"role": "user", "content": content},
    #         {
    #             "role": "assistant",
    #             "content": chat_completion.choices[0].message.content,
    #         },
    #     ]
    # )
    return reward, chat_completion


for future in asyncio.as_completed(
    eval(game, prompt, lowercase)
    for game, (prompt, lowercase) in zip(
        eval_games,
        cycle((prompt, lowercase) for prompt in prompts for lowercase in [True, False]),
    )
):
    try:
        reward, chat_completion = await future
        total_reward += reward
        # total_tokens += tokens.numel()
        pbar.update(1)
        update_postfix()
    except BaseException as e:
        exceptions.append(e)
        update_postfix()
pbar.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
100%|██████████| 20/20 [00:02<00:00,  9.80it/s, reward=0.775, average tokens=0, generated tokens=0, spend=$0.00, exceptions=0]


In [8]:
black_print(chat_completion)

ChatCompletion(
    id="65d5c157-a187-473f-8814-e7f505fac564",
    choices=[
        Choice(
            finish_reason="stop",
            index=0,
            logprobs=ChoiceLogprobs(
                content=[
                    ChatCompletionTokenLogprob(
                        token="<think>",
                        bytes=[60, 116, 104, 105, 110, 107, 62],
                        logprob=-1.2e-07,
                        top_logprobs=[
                            TopLogprob(
                                token="<think>",
                                bytes=[60, 116, 104, 105, 110, 107, 62],
                                logprob=-1.2e-07,
                                token_id=128798,
                            ),
                            TopLogprob(
                                token="<｜place▁holder▁no▁794｜>",
                                bytes=[
                                    60,
                                    239,
                                    

In [55]:
token_ids = tokenizer.encode(
    [
        {
            "role": "user",
            "content": "Hello, world!",
        },
        {
            "role": "assistant",
            "content": chat_completion.choices[0].message.content,
        },
    ],
    # Remove template logic that strips reasoning content from the chat messages
    chat_template=tokenizer.chat_template().replace(
        "{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}",
        "",
    ),
    add_generation_prompt=False,
    continue_final_message=True,
)
print(tokenizer.decode(token_ids))
token_ids

<｜begin▁of▁sentence｜><｜User｜>Hello, world!<｜Assistant｜><think>
Okay, let's tackle this word grouping puzzle. The task is to group the 16 words into four groups of four, each fitting a specific category, which could involve wordplay like palindromes, homophones, adding/dropping letters, etc. instead of just their meanings. The words provided are:

GREY, CUP, BILL, RIBBON, QUAD, BEAR, HOUSE, LAT, QUINN, TRI, MEDAL, COMMANDER, BROWN, TROPHY, PEC, HOWSER.

First, let me list all the words to see if any patterns jump out.

Looking for obvious categories. Colors? We have GREY and BROWN. Maybe a color group. So that's two words. But maybe more. Let me check others—no obvious color names beyond these two. So perhaps "GREY, BROWN" is part of a color group, but need two more. Wait, maybe other words can relate to colors through wordplay. For example, BEAR could be part of colors if thinking like "bear" as in "brown bear," but that's more about meaning. But the instruction says categories might u

tensor([151646, 151644,   9707,  ...,  43539,   3307,  31981])